In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
# Set default plotly template
import plotly.io as pio
pio.templates.default = "plotly_white"

import plotly.express as px

STATE = 'thermal'
IMG_WIDTH = 600
IMG_HEIGHT = 350

# Set default plotly image format and dimensions
pio.kaleido.scope.default_format = "pdf"
#pio.kaleido.scope.mathjax= None
pio.kaleido.scope.default_width = IMG_WIDTH
pio.kaleido.scope.default_height = IMG_HEIGHT

STYLES = {
    'plotly': (
        '#636EFA',
        '#EF553B',
        '#00CC96',
    ),
    'pastel': (
        '#66C5CC',
        '#F6CF71',
        '#F89C9C',
        '#DCB0F2',
    ),
    'pygal': (
        '#F44336',  # 0
        '#3F51B5',  # 4
        '#009688',  # 8
        '#FFC107',  # 13
        '#FF5722',  # 15
        '#9C27B0',  # 2
        '#03A9F4',  # 6
        '#8BC34A',  # 10
        '#FF9800',  # 14
        '#E91E63',  # 1
        '#2196F3',  # 5
        '#4CAF50',  # 9
        '#FFEB3B',  # 12
        '#673AB7',  # 3
        '#00BCD4',  # 7
        '#CDDC39',  # 11b
        '#9E9E9E',  # 17
        '#607D8B',  # 18
    )
}
LINESTYLES = [':', '--', '-']

mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=STYLES['plotly'], linestyle=LINESTYLES)
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['axes.grid'] = True
mpl.rcParams['grid.color'] = 'lightgrey'
mpl.rcParams['grid.linestyle'] = '--'
mpl.rcParams['grid.linewidth'] = 0.5
mpl.rcParams['xtick.color'] = 'lightgrey'
mpl.rcParams['ytick.color'] = 'lightgrey'
mpl.rcParams['xtick.labelcolor'] = 'black'
mpl.rcParams['ytick.labelcolor'] = 'black'
mpl.rcParams['axes.edgecolor'] = 'lightgrey'
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = 14
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['savefig.format'] = 'eps'

# Create saving folder if it does not exists
if not os.path.exists(f'../img/obs_evolution/{STATE}'):
    os.makedirs(f'../img/obs_evolution/{STATE}')

In [ ]:
df = pd.read_pickle(f'../objects/{STATE}/observables.pkl')
df = df[df['Time'] <= 1500]
df.head()

## Mean Photon Numbers

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(IMG_WIDTH/100, IMG_HEIGHT/100))
for i, (name, group) in enumerate(df.groupby('Interaction Time')):
    axs[i].plot(group['Time'], group['N1'], label='$S_1$')
    axs[i].plot(group['Time'], group['N2'], label='$S_2$')
    axs[i].set_xlabel('Time (a.u.)')
    axs[i].set_ylabel('Mean Photon Number')
    axs[i].set_title(f'Interaction Time = {name}')
    axs[i].legend()
plt.show()

## Entropies

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(IMG_WIDTH/100 + 3, IMG_HEIGHT/100), layout='tight')
ax_names = {
    0.05: r'$\Delta t = 0.05$',
    0.1: r'$\Delta t = 0.1$',
    1.0: r'$\Delta t = 1.0$',
}

y_names = {
    'S': r'$\mathcal{S}_V$',
    'S1': r'$\mathcal{S}_{V1}$',
    'S2': r'$\mathcal{S}_{V2}$'
}

for i, (name, group) in enumerate(df.groupby('Interaction Time')):
    for j, entropy in enumerate(['S', 'S1', 'S2']):
        axs[j].plot(group['Time'], group[entropy], label=f'{ax_names[name]}')
        axs[j].set_xlabel('Time (a.u.)')
        axs[j].set_ylabel(f'{y_names[entropy]}')

axs[0].legend()

plt.show()
fig.savefig(f'../img/obs_evolution/{STATE}/entropy_evolution.eps', format='eps')

## Quantum Discord

In [ ]:
fig, ax = plt.subplots(figsize=(IMG_WIDTH/100, IMG_HEIGHT/100))

for name, group in df.groupby('Interaction Time'):
    ax.plot(group['Time'], group['Quantum Discord'], label=f'{ax_names[name]}')
ax.set_ylabel('$\mathcal{D}$')
ax.legend()

ax.set_xlim(0, 600)

plt.show()

fig.savefig(f'../img/obs_evolution/{STATE}/discord_evolution.eps', format='eps')

## Mutual Information

In [ ]:
fig, ax = plt.subplots(figsize=(IMG_WIDTH/100, IMG_HEIGHT/100))

for i, (name, group) in enumerate(df.groupby('Interaction Time')):
    ax.plot(group['Time'], group['Mutual Information'], label=ax_names[name])
ax.set_ylabel('$\mathcal{I}$')
ax.legend()

ax.set_xlim(0, 600)

plt.show()

fig.savefig(f'../img/obs_evolution/{STATE}/mutinf_evolution.eps', format='eps')

## One-way Classical Correlations

In [ ]:
fig, ax = plt.subplots(figsize=(IMG_WIDTH/100, IMG_HEIGHT/100))

for i, (name, group) in enumerate(df.groupby('Interaction Time')):
    ax.plot(group['Time'], group['Mutual Information'] - group['Quantum Discord'], label=ax_names[name])
ax.set_ylabel('$\mathcal{J}$')
ax.legend()

ax.set_xlim(0, 600)

plt.show()

fig.savefig(f'../img/obs_evolution/{STATE}/mutinf_evolution.eps', format='eps')

## Logarithmic Negativity

In [ ]:
fig, ax = plt.subplots(figsize=(IMG_WIDTH/100, IMG_HEIGHT/100))
for name, group in df.groupby('Interaction Time'):
    ax.plot(group['Time'], group['Logarithmic Negativity'], label=f'{ax_names[name]}')

## Heat Exchanges

In [ ]:
df_to_plot = df.loc[df['Time'] > 0]
fig, ax = plt.subplots(figsize=(3.5, 4.5))
for i, (name, group) in enumerate(df_to_plot.groupby('Interaction Time')):
    ax.plot(group['Time'], group['Jc'], label=ax_names[name])
ax.set_xlim([0, 100])
ax.set_ylim([-0.05, 0.00001])
ax.set_xlabel('Time (a.u.)')
ax.set_ylabel('$J_c$')
ax.legend(loc='center right', title='')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(3.5, 4.5))
for i, (name, group) in enumerate(df_to_plot.groupby('Interaction Time')):
    ax.plot(group['Time'], group['J1'], label=ax_names[name])
ax.set_xlim([0, 1000])
ax.set_ylim([-0.0002, 0.000005])
ax.set_xlabel('Time (a.u.)')
ax.set_ylabel('$J_1$')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(3.5, 4.5))
linestyles = ['-', '--', '-.', ':']
for i, (name, group) in enumerate(df_to_plot.groupby('Interaction Time')):
    ax.plot(group['Time'], group['J2'], label=name, linestyle=linestyles[i % len(linestyles)])
ax.set_xlim([0, 1000])
ax.set_ylim([-0.004, 0.0001])
ax.set_xlabel('Time (a.u.)')
ax.set_ylabel('$J_2$')

# Create an inset plot
axins = fig.add_axes([0.43, 0.25, 0.4, 0.45])
for i, (name, group) in enumerate(df_to_plot.groupby('Interaction Time')):
    axins.plot(group['Time'], group['J2'], label=name, linestyle=linestyles[i % len(linestyles)])
axins.set_xlim([0, 1000])
axins.set_ylim([-0.00005, 0.00001])

plt.show()

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(IMG_WIDTH/100 + 2, IMG_HEIGHT/100), layout='constrained')
ax_names = {
    0.05: '$\Delta t = 0.05$',
    0.1: '$\Delta t = 0.1$',
    1.0: '$\Delta t = 1.0$',
}

y_names = {
    'Jc': '$J_c$',
    'J1': '$J_1$',
    'J2': '$J_2$'
}

for i, (name, group) in enumerate(df.loc[df['Time'] > 0].groupby('Interaction Time')):
    for j, current in enumerate(['Jc', 'J1', 'J2']):
        axs[j].plot(group['Time'], group[current], label=ax_names[name])
        axs[j].set_xlabel('Time (a.u.)')
        axs[j].set_ylabel(y_names[current])
for ax in axs:
    ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    ax.yaxis.get_offset_text().set_color('black')
        
axs[0].set_xlim([0, 1000])
axs[1].set_xlim([0, 1000])
axs[2].set_xlim([0, 200])
axs[0].set_ylim([-2e-4, 1e-5])
axs[1].set_ylim([-5e-5, 2e-6])
axs[2].set_ylim([-4e-3, 2e-4])
axs[0].legend()

# Create an inset plot
axins = fig.add_axes([0.9, 0.3, 0.15, 0.4])
for spine in axins.spines.values():
    spine.set_color('black')

for i, (name, group) in enumerate(df_to_plot.groupby('Interaction Time')):
    axins.plot(group['Time'], group['J2'], label=name)
axins.set_xlim([0, 1001])
axins.set_ylim([-0.00005, 0.00001])
axins.yaxis.get_offset_text().set_color('black')

plt.show()

fig.savefig(f'../img/obs_evolution/{STATE}/heat_evolution.eps', format='eps', bbox_inches='tight')